# Plotting and Analysis

The role of this notebook is to plot and analyze logs results of a run (or runs) of a simulator, given some fixed timing configuration.
These logs (bboxes.csv) are obtained by running a simulator on some experiments. The goal of these plots is to analyze worm's behavior,
and to analyze the systems error and how it's affected by different behaviors the worm exhibits.

It's important to note that for proper analysis, all the experiments that are analyzed by this notebook *at once* must have the same timing configuration (TimingConfig) parameters.

In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from wtracker.eval import *
from wtracker.sim.config import TimingConfig
from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.path_utils import join_paths

### Timing configuration and log files selection

In [ ]:
################################ User Input ################################

log_folder_path = None

############################################################################


if log_folder_path is None:
    log_folder_path = UserPrompt.open_directory(title="Select log directory to analyze")

log_file = join_paths(log_folder_path, "bboxes.csv")
time_config_path = join_paths(log_folder_path, "time_config.json")
analysis_save_path = join_paths(log_folder_path, "analyzed.csv")

print("Base directory: ", log_folder_path)
print("Log file: ", log_file)
print("Time config file: ", time_config_path)
print("Analysis save file: ", analysis_save_path)

In [ ]:
from pprint import pprint

timing_config = TimingConfig.load_json(time_config_path)
analyzer = DataAnalyzer.load(timing_config, log_file)

pprint(timing_config)

### Analyze log data

In [ ]:
analyzer.initialize(period=10)

analyzer.clean(
    trim_cycles=True,
    imaging_only=True,
    bounds=None,
)

analyzer.change_unit("sec")

In [ ]:
analyzer.calc_anomalies(
    no_preds=True,
    min_bbox_error=np.inf,
    min_dist_error=np.inf,
    min_speed=np.inf,
    min_size=np.inf,
    remove_anomalies=True,
)

In [ ]:
analyzer.print_stats()

In [ ]:
analyzer.describe(
    columns=["wrm_speed", "bbox_error", "worm_deviation"],
    percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.97, 0.98, 0.99],
)

In [ ]:
analyzer.save(analysis_save_path)

### Plotting

Notice that all of below plots accept `condition` as a parameter.
`condition` is expected to be a function of the following signature:

```python
def cond_func1(input_df: pd.DataFrame) -> pd.DataFrame:
    return (input_df["wrm_speed"] > 5) &  (input_df["wrm_speed"] <= 30)
```

In python, such functions can be also declared without an explicit name and declaration, using the following syntax:
(for more information read about lambda functions)

```python
cond_func1 = lambda input_df: (input_df["wrm_speed"] > 5) & (input_df["wrm_speed"] <= 30)
cond_func2 = lambda input_df: input_df["phase"] == "imaging"
```

In [ ]:
# print column names of the data
pprint([f"{i}: {col}" for i, col in enumerate(analyzer.column_names())])

In [ ]:
# create the plotter
pltr = Plotter([analyzer.data], plot_height=7, palette="bright")

In [ ]:
pltr.plot_trajectory()
plt.show()

In [ ]:
pltr.plot_head_size()
plt.show()

In [ ]:
pltr.plot_speed(
    log_wise=True,
    condition=lambda x: x["wrm_speed"] <= 800,
    aspect=0.5,
)
plt.show()

In [ ]:
pltr.plot_speed_vs_error(
    error_kind="bbox",
    cycle_wise=True,
    condition=lambda df: (df["wrm_speed"] < 1000) & (df["bbox_error"] > 1e-5),
)
plt.show()

In [ ]:
pltr.plot_speed_vs_error(
    error_kind="dist",
    cycle_wise=True,
    condition=lambda df: (df["wrm_speed"] < 1000) & (df["worm_deviation"] < 300),
)
plt.show()

In [ ]:
pltr.plot_deviation(
    percentile=0.995,
    kind="boxen",
    k_depth="proportion",
    outlier_prop=0.02,
    saturation=0.5,
)
plt.show()